In [1]:
import os
import sys

os.chdir('/Users/mask/Documents/Проеты_2025/gpt_4_mini_hackaton/app_talebot_v3')
# Получаем текущую рабочую директорию
current_dir = os.getcwd()

# Добавляем родительскую директорию текущей директории в путь Python
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [3]:
import aiohttp
import asyncio


class HeadHunterAPI:
    def __init__(self, client_id: str, client_secret: str, redirect_uri: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.redirect_uri = redirect_uri
        self.base_url = "https://hh.ru"
        self.token_url = f"{self.base_url}/oauth/token"
        self.auth_url = f"{self.base_url}/oauth/authorize"
        self._access_token = None

    def get_auth_url(self) -> str:
        return (
            f"{self.auth_url}?response_type=code"
            f"&client_id={self.client_id}"
            f"&redirect_uri={self.redirect_uri}"
        )

    async def exchange_code_for_tokens(self, code: str):
        data = {
            "grant_type": "authorization_code",
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "code": code,
            "redirect_uri": self.redirect_uri,
        }
        async with aiohttp.ClientSession() as session:
            async with session.post(self.token_url, data=data) as response:
                if response.status == 200:
                    tokens = await response.json()
                    self._access_token = tokens["access_token"]
                    print("Токен успешно получен!")
                else:
                    error_data = await response.text()
                    print(f"Ошибка при получении токена: {error_data}")

    async def make_api_request(
        self,
        endpoint: str,
        method: str = "POST",
        data: dict = None,
        params: dict = None,
    ):
        if not self._access_token:
            raise ValueError("Токен доступа отсутствует. Необходимо авторизоваться.")

        headers = {
            "Authorization": f"Bearer {self._access_token}",
            "User-Agent": "HeadHunterAPI/1.0",
        }
        url = f"{self.base_url}{endpoint}"

        async with aiohttp.ClientSession() as session:
            if method == "POST":
                async with session.post(url, headers=headers, json=data) as response:
                    response_text = await response.text()
                    print(f"Ответ сервера: {response_text}")  # Для диагностики
                    if response.status == 403:
                        raise PermissionError("Ошибка 403: недостаточно прав.")
                    response.raise_for_status()
                    return await response.json()


In [4]:
async def authorize_client():
    client_id = "J58PIDNM3UQCNE85UVRMSSIO7F8HUNGD7KPNR9004IK3LIKE6GNBAPDBIDHMOUH1"
    client_secret = "IGBU6EJV14587HUCAAJRRC0B63SNSUDL7J2PR39RR438SPECU64LAQ15EVNEJLL2"
    redirect_uri = "https://gpt4o-mini-hackathon.onrender.com"

    hh_client = HeadHunterAPI(client_id, client_secret, redirect_uri)

    # Получение ссылки для авторизации
    auth_url = hh_client.get_auth_url()
    print(f"Пройдите по ссылке для авторизации: {auth_url}")

    # Ввод кода авторизации
    code = input("Введите полученный код: ")
    await hh_client.exchange_code_for_tokens(code)

    return hh_client


# Запуск авторизации
hh_client = await(authorize_client())


Пройдите по ссылке для авторизации: https://hh.ru/oauth/authorize?response_type=code&client_id=J58PIDNM3UQCNE85UVRMSSIO7F8HUNGD7KPNR9004IK3LIKE6GNBAPDBIDHMOUH1&redirect_uri=https://gpt4o-mini-hackathon.onrender.com
Ошибка при получении токена: {"error":"invalid_grant","error_description":"code has already been used"}


In [4]:
async def create_resume(hh_client):
    resume_data = {
        "first_name": "Павел",
        "last_name": "Дуров",
        "middle_name": "Валерьевич",
        "title": "Руководитель разработки программного обеспечения",
        "birth_date": "1984-10-10",
        "contact": [
            {
                "preferred": True,
                "type": {"id": "email", "name": "Эл. почта"},
                "value": "pavel.durov@example.dev"
            },
            {
                "preferred": False,
                "type": {"id": "phone", "name": "Телефон"},
                "value": "+7 999 999 9999"
            }
        ],
        "skills": (
            "Я опытный руководитель разработки программного обеспечения с 18-летним стажем в технологической отрасли. "
            "Эксперт в управлении удаленными и офисными командами. Специалист в языках программирования: Ruby, Python, JavaScript, "
            "TypeScript, Elixir. Опыт в области data science и машинного обучения позволяет успешно внедрять масштабируемые AI-решения."
        ),
        "experience": [
            {
                "company": "Яндекс",
                "position": "Директор разработки программного обеспечения (удалённо)",
                "start": "2022-01-01",
                "end": None,
                "description": (
                    "- Я превратил инженерное подразделение в ключевой источник дохода для Data Kernel, быстро расширив клиентскую базу с момента ее создания до надежного сообщества."
                    "- Возглавил интеграцию передовых технологий искусственного интеллекта и использование масштабируемых векторных баз данных, что значительно расширило возможности продукта и его позиционирование на рынке."
                    "- Под моим руководством команда добилась существенного роста в развитии навыков, сосредоточив внимание на достижении стратегических целей проекта, которые существенно повлияли на направление развития компании."
                    "- Также сыграл решающую роль в определении долгосрочных стратегических инициатив компании, особенно в области внедрения технологий искусственного интеллекта, которые установили новые стандарты в отрасли."
                )
            },
            {
                "company": "СберТех",
                "position": "Старший менеджер разработки программного обеспечения (удалённо)",
                "start": "2019-01-01",
                "end": "2022-01-01",
                "description": (
                    "- Руководил разработкой стратегии и операционной деятельностью в тесном сотрудничестве с руководителями высшего звена, играя ключевую роль в формировании технологической траектории компании."
                    "- Руководил различными командами, работающими в разных часовых поясах в Северной Америке и Европе, создавая атмосферу прозрачности и взаимного уважения, что повышало производительность и моральный дух команды."
                    "- Мои инициативы в области подбора персонала, наставничества и удержания талантливых специалистов сыграли решающую роль в формировании культуры постоянного совершенствования и высокой производительности."
                )
            },
            {
                "company": "VK",
                "position": "Основатель и генеральный директор (удалённо)",
                "start": "2019-01-01",
                "end": "2022-01-01",
                "description": (
                    "- Я основал Innovapet, стартап, специализирующийся на инновационных решениях Интернета вещей для ухода за домашними животными, включая революционный ошейник с GPS-трекингом, который значительно повысил безопасность питомца и спокойствие владельца."
                    "- Так же отвечал за разработку продукта от концепции до воплощения в жизнь, тесно сотрудничая с инженерными командами и партнерами по маркетингу, чтобы обеспечить успешный выход на рынок."
                    "- Успешно создал консультативный совет, открыл производственные мощности за рубежом и провел компанию через успешный начальный этап финансирования, продемонстрировав свои лидерские качества и предпринимательскую хватку."
                    "- Создана начальная версия продукта с использованием MongoDB"
                )
            },
            {
                "company": "Mail.ru",
                "position": "Менеджер инженерного отдела (удалённо)",
                "start": "2018-01-01",
                "end": "2019-01-01",
                "description": (
                    "- Я играл важную роль в разработке и реализации стратегических планов, которые укрепили межведомственную координацию и доверие, что привело к улучшению результатов проекта."
                    "- Так же руководил несколькими инженерными командами, способствуя формированию культуры, сочетающей продуктивность с инновациями, и внедрил систему постановки целей, которая соответствовала долгосрочным целям компании."
                    "- Являлся очень практичным менеджером, использующим ruby on rails и react для создания нового продукта."
                )
            },
            {
                "company": "Лаборатория Касперского",
                "position": "Менеджер инженерного отдела (удалённо)",
                "start": "2016-01-01",
                "end": "2018-01-01",
                "description": (
                    "- Руководил формированием и развитием высокопроизводительной инженерной команды, которая сыграла ключевую роль в увеличении выручки компании на 500% в течение двух лет."
                    "- Его лидерство в области интеграции анализа данных в процессы принятия бизнес-решений привело к разработке инструмента прогнозного моделирования, который произвел революцию в анализе поведения клиентов."
                )
            },
            {
                "company": "Озон",
                "position": "Старший разработчик программного обеспечения (удалённо)",
                "start": "2015-01-01",
                "end": "2016-01-01",
                "description": (
                    "Разработал и оптимизировал центральный API, значительно улучшив его функциональность. "
                    "Внедрил стратегию кеширования, что позволило существенно сократить время отклика и снизить нагрузку на систему."
                )
            },
            {
                "company": "Тинькофф",
                "position": "Технический директор (CTO)",
                "start": "2013-01-01",
                "end": "2015-01-01",
                "description": (
                    "Руководил технологической стратегией компании после получения финансирования серии A. "
                    "Руководил разработкой продукта с использованием React и Angular, значительно улучшив производительность платформы."
                )
            }
        ],
        "education": {
            "level": {"id": "higher", "name": "Высшее"},
            "primary": [
                {"name": "МГУ им. М.В. Ломоносова", "result": "Диплом с отличием"},
                {"name": "Курс: Специализация Data Science", "result": "Сертификат (Coursera, Университет Джона Хопкинса)"},
                {"name": "MBA: Информационные технологии", "result": "Лондонская школа бизнеса"}
            ]
        }
    }

    try:
        response = await hh_client.make_api_request("/resumes", method="POST", data=resume_data)
        print("Резюме создано успешно:", response)
    except Exception as e:
        print("Ошибка при создании резюме:", e)


In [6]:
async def check_token(hh_client):
    try:
        response = await hh_client.make_api_request("/me", method="GET")
        print("Проверка токена: успешна", response)
    except Exception as e:
        print("Ошибка проверки токена:", e)

In [ ]:
# Запуск создания резюме
await (create_resume(hh_client))

In [2]:
client_id = "J58PIDNM3UQCNE85UVRMSSIO7F8HUNGD7KPNR9004IK3LIKE6GNBAPDBIDHMOUH1"
redirect_uri = "https://gpt4o-mini-hackathon.onrender.com"

auth_url = (
    f"https://hh.ru/oauth/authorize?response_type=code"
    f"&client_id={client_id}"
    f"&redirect_uri={redirect_uri}"
)
print(auth_url)


https://hh.ru/oauth/authorize?response_type=code&client_id=J58PIDNM3UQCNE85UVRMSSIO7F8HUNGD7KPNR9004IK3LIKE6GNBAPDBIDHMOUH1&redirect_uri=https://gpt4o-mini-hackathon.onrender.com


In [9]:
import requests

client_id = client_id
client_secret = "IGBU6EJV14587HUCAAJRRC0B63SNSUDL7J2PR39RR438SPECU64LAQ15EVNEJLL2"
redirect_uri = redirect_uri
authorization_code = "GFOCBUTC8UCUKG9OFUIJPS86HETKHMTDGHQV319BJD8R3CEQU99D6CIORK8L9TJ7"

token_url = "https://hh.ru/oauth/token"
data = {
    "grant_type": "authorization_code",
    "client_id": client_id,
    "client_secret": client_secret,
    "code": authorization_code,
    "redirect_uri": redirect_uri,
}

response = requests.post(token_url, data=data)
tokens = response.json()
access_token = tokens.get("access_token")


In [10]:
print(access_token)

None


In [15]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "User-Agent": "YourAppName/1.0",
}

response = requests.get("https://api.hh.ru/me", headers=headers)
user_info = response.json()
print(user_info)


{'auth_type': 'applicant', 'is_applicant': True, 'is_employer': False, 'is_admin': False, 'is_hiring_manager': False, 'is_application': False, 'is_employer_integration': False, 'crypted_id': '396A0B81BD0C4F5B44DA5BC9BB260091A5471C6F722104504734B32F0D03E9F8', 'id': '13055057', 'is_anonymous': False, 'email': 'angers07@mail.ru', 'first_name': 'Павел', 'middle_name': 'Валерьевич', 'last_name': 'Дуров', 'resumes_url': 'https://api.hh.ru/resumes/mine', 'negotiations_url': 'https://api.hh.ru/negotiations', 'is_in_search': True, 'mid_name': 'Валерьевич', 'employer': None, 'manager': None, 'phone': '79875056990', 'counters': {'new_resume_views': 3, 'unread_negotiations': 0, 'resumes_count': 3}, 'profile_videos': {'items': []}, 'personal_manager': None}


In [6]:
resume_data = {
        "first_name": "Павел",
        "last_name": "Дуров",
        "middle_name": "Валерьевич",
        "title": "Руководитель разработки программного обеспечения",
        "birth_date": "1984-10-10",
        "contact": [
            {
                "preferred": True,
                "type": {"id": "email", "name": "Эл. почта"},
                "value": "pavel.durov@example.dev"
            },
            {
                "preferred": False,
                "type": {"id": "cell", "name": "Мобильный телефон"},
                "value": {
                    "city": "955",
                    "country": "7",
                    "formatted": "7-955-999-9999",
                    "number": "9999999"
                },
                "verified": False
            }
        ],
        "skills": (
            "Павел Дуров — опытный руководитель разработки программного обеспечения с 18-летним стажем в технологической отрасли. "
            "Эксперт в управлении удаленными и офисными командами. Специалист в машинном обучения."
        ),
        "experience": [
            {
                "company": "Яндекс",
                "position": "Директор AI-направления (удалённо)",
                "start": "2022-01-01",
                "end": None,
                "description": (
                    "Внедрил продвинутую аналитику для голосовых и текстовых данных, сократив время анализа вдвое. "
                    "Управлял распределённой командой (инженеры, аналитики), обеспечив эффективную коммуникацию. "
                    "Разработал внутренние инструменты для автоматизации сбора и обработки данных."
                )
            },
            {
                "company": "СберТех",
                "position": "Руководитель ML-команды (удалённо)",
                "start": "2019-01-01",
                "end": "2022-01-01",
                "description": (
                    "Интегрировал модели машинного обучения в инфраструктуру компании, расширив возможности персонализации. "
                    "Спроектировал модульную архитектуру, упрощающую подключение новых AI-решений. "
                    "Установил ключевые метрики эффективности (KPI) для data-проектов совместно с бизнес-отделом."
                )
            },
            {
                "company": "VK",
                "position": "Основатель и генеральный директор (удалённо)",
                "start": "2019-01-01",
                "end": "2022-01-01",
                "description": (
                    "Основал VK, стартап, специализирующийся на инновационных решениях Интернета вещей. Инициировал использование гибких методологий (Scrum, Kanban) для быстрой разработки AI-функционала. "
                    "Создал модели прогнозной аналитики, оптимизировав рекомендательные системы. "
                    "Организовал программу наставничества, повысив общий уровень знаний команды в ML."
                )
            },
        ],
        "education": {
            "level": {"id": "higher", "name": "Высшее"},
            "primary": [
                {
                    "name": "МГУ им. М.В. Ломоносова",
                    "organization": "Факультет прикладной математики",
                    "result": "Математик-программист",
                    "year": 2006  # Год окончания обязателен
                },
                {
                    "name": "Курс: Специализация Data Science",
                    "organization": "Coursera, Университет Джона Хопкинса",
                    "result": "Сертификат",
                    "year": 2018  # Добавьте год окончания
                },
                {
                    "name": "MBA: Информационные технологии",
                    "organization": "Лондонская школа бизнеса",
                    "result": "Магистр",
                    "year": 2020  # Добавьте год окончания
                }
            ],
            "additional": None,
            "attestation": None,
            "elementary": None
        }
    }

In [7]:
import requests

# URL для создания резюме
create_resume_url = "https://api.hh.ru/resumes"

# Заголовки запроса
headers = {
    "Authorization": f"Bearer {access_token}",
    "User-Agent": "YourAppName/1.0",
    "Content-Type": "application/json",
}

# Отправка POST-запроса для создания резюме
response = requests.post(create_resume_url, headers=headers, json=resume_data)

# Проверка ответа
if response.status_code == 201:
    print("Резюме успешно создано!")
    resume_info = response.json()
    print("ID резюме:", resume_info.get("id"))
else:
    print("Ошибка при создании резюме:", response.status_code)
    print("Детали:", response.json())


Резюме успешно создано!


JSONDecodeError: Expecting value: line 1 column 1 (char 0)